<a href="https://colab.research.google.com/github/S-Ali-S/NLP/blob/main/NaiveBayes_NLP_Sharifian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [105]:
import tensorflow_datasets as tfds
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from collections import Counter


nltk.download('punkt')
nltk.download('stopwords')

(train_data,test_data)=tfds.load('imdb_reviews',split=['train','test[:40%]'],as_supervised=True)

tokenizer = RegexpTokenizer(r'\w+')
stop_words = set(stopwords.words('english'))


def tokenize_the_text(text):
  text2lower=text.lower()
  tokens= tokenizer.tokenize(text2lower)
  tokens= [token for token in tokens if token not in stop_words]

  return tokens


positive_samples=[
    (tokenize_the_text(text.numpy().decode('utf-8')) ,label.numpy())
     for text ,label in train_data
        if label.numpy()==1
    ]

negative_samples=[
    (tokenize_the_text(text.numpy().decode('utf-8')),label.numpy())
     for text ,label in train_data
        if label.numpy()==0
    ]


positive_words = Counter()
for tokens, _ in positive_samples[:100]:
    positive_words.update(tokens)


negative_words = Counter()
for tokens, _ in negative_samples[:100]:
    negative_words.update(tokens)


positive_words_probability=Counter()
negative_words_probability=Counter()


total_positive_count = sum(positive_words.values())
total_negative_count = sum(negative_words.values())

for token in positive_words:
  positive_words_probability[token]=positive_words[token]/total_positive_count

for token in negative_words:
  negative_words_probability[token]=negative_words[token]/total_negative_count



def calcuate_proability_of_sentence(text):
  total_prob_of_sentence=0.5 # length of a sample divided by the sum of the both(100/100+100)
  negative_prob=1 * total_prob_of_sentence
  positive_prob=1 * total_prob_of_sentence

  for token in tokenize_the_text(text):
    if positive_words_probability[token]==0 or positive_words_probability[token] is None or negative_words_probability[token]==0 or negative_words_probability[token] is None :
      negative_prob *= 1
      positive_prob *= 1
    else:
      negative_prob*= negative_words_probability[token]
      positive_prob*= positive_words_probability[token]


  return positive_prob, negative_prob



test_data = test_data.take(20)

true_positives = 0
true_negatives = 0
false_positives = 0
false_negatives = 0

for text, label in test_data:
    positive_prob, negative_prob = calcuate_proability_of_sentence(text.numpy().decode('utf-8'))


    predicted_label = 1 if positive_prob > negative_prob else 0


    if predicted_label == 1 and label.numpy() == 1:
        true_positives += 1
    elif predicted_label == 0 and label.numpy() == 0:
        true_negatives += 1
    elif predicted_label == 1 and label.numpy() == 0:
        false_positives += 1
    elif predicted_label == 0 and label.numpy() == 1:
        false_negatives += 1

    print(label)

precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) != 0 else 0
recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) != 0 else 0
f_measure = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0
accuracy = (true_positives + true_negatives) / (true_positives + true_negatives + false_positives + false_negatives)

print("Precision:", precision)
print("Recall:", recall)
print("F-measure:", f_measure)
print("Accuracy:", accuracy)



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
